# Imports

In [1]:
%pip install matplotlib
%pip install scikit-learn
%pip install scipy
%pip install tensorflow
%pip install numpy
%pip install pandas
%pip install setuptools

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf

from tensorflow.image import resize
from tensorflow.keras.backend import clear_session
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.metrics import  Recall, CategoricalAccuracy
from IPython.display import clear_output

from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import concatenate as concat
from scipy.stats import entropy
import os

from helpers.help import *


np.random.seed(0)

# Start

In [3]:
# Open Diabetic Retinopathy dataset
path = os.path.join(os.getcwd(),'gaussian_ds')
label_dict={'Mild':1,'Moderate':1,'Proliferate_DR':1,'Severe':1,'No_DR':0}

# remove macOS file
folders = os.listdir(path)
# folders.remove('.DS_Store')

# get all the samples
array = []
for i in folders:
    detailPath = os.path.join(path,i)
    for j in os.listdir(detailPath):
        array.append([os.path.join(detailPath,j),label_dict[i.split('.')[0]]])

# transforms the array into nparray
dataset=np.array(array)

np.size(dataset,0)

dataset[0:3]

array([['/Users/leonardosousa/Desktop/mestrado/IA/Delivery/code/RetinopathyAlgorithm/gaussian_ds/Mild/2d7666b8884f.png',
        '1'],
       ['/Users/leonardosousa/Desktop/mestrado/IA/Delivery/code/RetinopathyAlgorithm/gaussian_ds/Mild/50840c36f0b4.png',
        '1'],
       ['/Users/leonardosousa/Desktop/mestrado/IA/Delivery/code/RetinopathyAlgorithm/gaussian_ds/Mild/30cab14951ac.png',
        '1']], dtype='<U119')

# Pre-Processing

In [4]:
# Get characteristics and labels (ensured they have the right type)
X,y=dataset[:,0],dataset[:,1]
y = y.astype(int)
y = to_categorical(y)

#Shuffle the dataset (to make a unbiased model)
p = np.random.permutation(len(X))
X,y = X[p], y[p]

#Strip off 10% samples for hold out test set
test_idxs = np.random.choice(len(X), size=int(0.1*len(X)), replace=False, p=None)
x_test, y_test = X[test_idxs],y[test_idxs]

#Delete the test set samples from X,y 
X = np.delete(X, test_idxs)
y = np.delete(y, test_idxs, axis = 0)

#usual train-val split. We use 11% here just match the test set size to validation set.
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.11)

# Separate the Seed and Pools
- **Seeds**: what is used to start the training
- **Pool**: serve as a pool for uncertainty sampling

In [5]:
initial_seed = 100
x_seed , x_pool = x_train[0:initial_seed], x_train[initial_seed:]
y_seed , y_pool = y_train[0:initial_seed], y_train[initial_seed:]

print(f"Samples in Seed set: {x_seed.shape[0]}")
print(f"Samples in Pool: {x_pool.shape[0]}")
print(f"Samples in Validation set: {x_val.shape[0]}")
print(f"Samples in Test set: {x_test.shape[0]}")

Samples in Seed set: 100
Samples in Pool: 2833
Samples in Validation set: 363
Samples in Test set: 366


# Converts into a dataset compatible with the model

In [6]:
#The buid_dataset is a custom function that returns tensor batches

val_dataset=build_dataset(x_val,y_val,repeat=False,batch=64)
test_dataset=build_dataset(x_test,y_test,repeat=False,batch=64)
pool_dataset=build_dataset(x_pool,y_pool,repeat=False,batch=64, shuffle = False)

BATCH_SIZE=32

STEPS_PER_EPOCH=len(x_train)/BATCH_SIZE

train_dataset=build_dataset(x_seed,y_seed,batch=BATCH_SIZE)
input_shape=train_dataset.element_spec[0].shape[1:]

# Model definition

In [7]:
model=simple_model(input_shape)
model.compile(
        loss = "categorical_crossentropy",
        optimizer = Adam(),
        metrics=[CategoricalAccuracy()]
    )
model.summary()

/Users/leonardosousa/Desktop/mestrado/IA/Delivery/code/RetinopathyAlgorithm/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 110, 110, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 110, 110, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 55, 55, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 55, 55, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 53, 53, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 53, 53, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,170,626 (42.61 MB)

 Trainable params: 11,169,922 (42.61 MB)

 Non-trainable params: 704 (2.75 KB)

In [8]:
# Saves the best model
checkpoint=ModelCheckpoint(filepath='model/model_al.keras',
                           monitor='val_loss',save_best_only=True,verbose=1)

# Logs the progress of training
csv_logger=keras.callbacks.CSVLogger('logger/trainlog_al.csv',
                                     separator=',',append=False)

# prevent overfitting
early_stopper=keras.callbacks.EarlyStopping(monitor='val_loss',
                                            min_delta=0.001,
                                            restore_best_weights=True,
                                            patience=10)

callbacks_list=[checkpoint,early_stopper,csv_logger]

# Model training
- **categorical_accuracy**: measures how accurately the model is predicting the right class for each sample across multiple classes
- **loss**: how well the model fits the training data; lower values are better.
- **val_categorical_accuracy**: On the validation set, the model’s accuracy we can see potential overfitting.
- **val_loss**: On the validation set, the loss.

In [9]:
# EPOCHS = 20 # minimalist
# EPOCHS = 200 # standard
EPOCHS = 100 # standard
model.fit(train_dataset,steps_per_epoch=int(STEPS_PER_EPOCH),epochs=EPOCHS,
          validation_data=val_dataset,validation_steps=None,
          callbacks=callbacks_list)

Epoch 1/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - categorical_accuracy: 0.9037 - loss: 0.3527
Epoch 1: val_loss improved from inf to 8.09535, saving model to model/model_al.keras
91/91 ━━━━━━━━━━━━━━━━━━━━ 115s 1s/step - categorical_accuracy: 0.9043 - loss: 0.3505 - val_categorical_accuracy: 0.4904 - val_loss: 8.0953
Epoch 2/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - categorical_accuracy: 0.9935 - loss: 0.0257
Epoch 2: val_loss improved from 8.09535 to 1.83726, saving model to model/model_al.keras
91/91 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - categorical_accuracy: 0.9936 - loss: 0.0257 - val_categorical_accuracy: 0.5840 - val_loss: 1.8373
Epoch 3/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - categorical_accuracy: 0.9965 - loss: 0.0096
Epoch 3: val_loss did not improve from 1.83726
91/91 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - categorical_accuracy: 0.9965 - loss: 0.0096 - val_categorical_accuracy: 0.6033 - val_loss: 2.0134
Epoch 4/100
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - categorical_accurac

# Initial Model Evaluation on Test Dataset (Smaller Dataset / Seeded)

In [10]:
model = keras.models.load_model("model/model_al.keras")
print("-" * 100)
print(model.evaluate(test_dataset, verbose=0,return_dict=True))

----------------------------------------------------------------------------------------------------
{'categorical_accuracy': 0.9016393423080444, 'loss': 0.5556147694587708}


In [11]:
al_history = []
csv_logger=keras.callbacks.CSVLogger('logger/trainlog_al.csv',
                                 separator=',',append=True)
callbacks_list=[checkpoint,early_stopper,csv_logger]


# Include baseline (full train) model

In [12]:
try:
    
    model_full = keras.models.load_model("model/model_baseline.keras")
    _, acc_baseline = model_full.evaluate(test_dataset)

    print("-" * 100)
    print(model_full.evaluate(test_dataset, verbose=0,return_dict=True))
    
except FileNotFoundError:
    print("model file model_baseline.h5 not found. Make sure to run 01_Training_Full.ipynb entirely")


6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 409ms/step - categorical_accuracy: 0.9467 - loss: 0.1614
----------------------------------------------------------------------------------------------------
{'categorical_accuracy': 0.9535518884658813, 'loss': 0.13201043009757996}


# Entering AL Loop
We will now iteratively query the pool for samples and add them to the seed set. Every time we can pick sampling_size number of points from the pool with the largest entropy.

Loop: 
- **Prediction on pool**: Uses the current model to predict in the pool dataset;
- **Uncertainty Sampling**: Selects *sampling_size* samples with the highest entropy (least confident predictions);
- **Acquire Samples**: Moves high-entropy samples from x_pool to x_seed to continue training;
- **Stopping Condition**: If the accuracy is close to the one in the baseline, the training stops;
- **Re-train**: Updates the model and continues training with the new samples in the seed set;

In [13]:
sampling_size=50
num_iterations = int(x_pool.shape[0]/sampling_size)


# num_iterations = num_iterations/10 # artificial stopper
num_iterations = int(x_pool.shape[0]/sampling_size)

print(sampling_size)


acc_baseline = round(acc_baseline,4)

# num_iterations = int(num_iterations -20) # reducing steps
for iteration in range(num_iterations):
    
    #Step_1
    loss, acc = model.evaluate(test_dataset, verbose=0)
    print(f"Test Set Accuracy after {iteration} iteration {acc}")
    al_history.append([loss, acc, x_seed.shape[0], x_pool.shape[0]])
    
    if acc_baseline - 0.0025 < acc <= acc_baseline + 0.0025:
        print("Terminating Training")
        break
    
    #Step_2
    #Use the current model to predict the pool dataset
    print("Predicting pool dataset")
    y_pool_proba = model.predict(pool_dataset)
    
    #Pick the index of the top entropy samples in pool
    pool_max_ents = np.argsort(entropy(y_pool_proba.T))[-sampling_size:]
    
    #Acquire those samples from pool
    x_sample = x_pool[pool_max_ents]
    y_sample = y_pool[pool_max_ents]
    
    #Add these samples to the seed dataset
    y_seed = concat((y_seed,y_sample),axis=0)
    x_seed = concat((x_seed,x_sample),axis=0)
     
    #Delete the acquired samples from pool
    x_pool = np.delete(x_pool, pool_max_ents, 0 )
    y_pool = np.delete(y_pool, pool_max_ents, 0 )

    #Build the tensorflow dataset object for this iteration
    pool_dataset = build_dataset(x_pool,y_pool,repeat=False,batch=256,
                                 shuffle = False)
    train_dataset = build_dataset(x_seed,y_seed,batch=BATCH_SIZE) 

    print(f"Samples in seed dataset {x_seed.shape[0]} , in pool dataset {x_pool.shape[0]}")
    print("-" * 100)

    #Step_3
    model.compile(
        loss = "binary_crossentropy",
        optimizer = Adam(),
        metrics=[CategoricalAccuracy()]
    )
    
    history = model.fit(train_dataset,steps_per_epoch=int(STEPS_PER_EPOCH),epochs=100,
          validation_data=val_dataset,validation_steps=None,
          callbacks=callbacks_list)
    
    #If the fit method generated a new best model , load it for
    #the next iteration
    model = keras.models.load_model("model/model_al.keras")
    clear_output()
    clear_session()

Test Set Accuracy after 12 iteration 0.9535518884658813
Terminating Training


In [14]:
df = pd.DataFrame(al_history, columns = ['Test Loss', 'Test Accuracy', 'Seed', 'Pool'])
df

,Test Loss,Test Accuracy,Seed,Pool
0,0.555615,0.901639,100,2833
1,0.429400,0.937158,150,2783
2,0.429400,0.937158,200,2733
3,0.388142,0.928962,250,2683
4,0.388142,0.928962,300,2633
5,0.208689,0.945355,350,2583
6,0.208689,0.945355,400,2533
7,0.208689,0.945355,450,2483
8,0.208689,0.945355,500,2433
9,0.199219,0.948087,550,2383


In [15]:
df.to_csv('logger/AL_tracking.csv', index = False)